<a href="https://colab.research.google.com/github/LucaForziati/DS3_coderhouse/blob/main/trabajo_final_redes_neuronales_ForziatiGangi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clasificador de flores**

El presente proyecto busca la creación de un sistema que sea capaz de clasificar diferentes flores.

Se toma el dataset "Flowers102" de los datasets disponibles en PyTorch.

# librerias

In [7]:
import torch
from torch import nn
import torchvision
from IPython import display
from torchvision import transforms
from torch.utils import data

import numpy as np

# cargar dataset

cargar el dataset que se extrae los datasets de torchvision.

In [8]:
# funcion que carga el dataset
def load_data_flowers102(batch_size, resize=28):
    # definir transformacion. Todas las imagenes tendran un tamaño de 28x28
    trans = transforms.Compose([
        transforms.Resize((resize, resize)),
        transforms.ToTensor()
    ])

    # cargar los datasets de entrenamiento y prueba
    flowers_train = torchvision.datasets.Flowers102(
        root="../data", split="train", transform=trans, download=True)
    flowers_test = torchvision.datasets.Flowers102(
        root="../data", split="test", transform=trans, download=True)

    # crear los dataloaders
    train_loader = data.DataLoader(flowers_train, batch_size,
                                 shuffle=True, num_workers=1)
    test_loader = data.DataLoader(flowers_test, batch_size,
                                shuffle=False, num_workers=1)

    return train_loader, test_loader

In [9]:
# definir el tamaño del batch
batch_size = 256

# cargar los dataset y guardarlos
train_iter, test_iter = load_data_flowers102(batch_size)

In [10]:
# Primero, definimos el tamaño de imagen y número de clases
image_size = 28  # Mantenemos 28x28 por simplicidad
num_classes = 102  # Flowers102 tiene 102 clases
input_size = image_size * image_size * 3  # * 3 porque son imágenes RGB

# Red Fully Connected

In [11]:
# crear la red neuronal
net = nn.Sequential( # cada capa se aplica una detras de la otra
    nn.Flatten(), # aplanar las imagenes
    nn.Linear(input_size, 256),  # Capa inicial, fully connected
    nn.ReLU(),                   # funcion de activacion ReLU
    nn.Linear(256, 128),         # Otra capa fully connected que reduce las caracteristicas de 256 -> 128
    nn.ReLU(),                   # Otra función de activación ReLU
    nn.Linear(128, num_classes)  # Capa de salida
)

# pesos de las capas fully connected
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=2352, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=128, bias=True)
  (4): ReLU()
  (5): Linear(in_features=128, out_features=102, bias=True)
)

In [12]:
# Parámetros de entrenamiento
# se define el tamaño del batch, la tasa de aprendizaje y el numero de epochs
batch_size, lr, num_epochs = 256, 0.1, 10
# funcion de perdida
loss = nn.CrossEntropyLoss(reduction='none')
# optimizador
trainer = torch.optim.SGD(net.parameters(), lr=lr)

In [13]:
# Función de precisión
def accuracy(y_hat, y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [14]:
# Bucle de entrenamiento
for epoch in range(num_epochs):
    # variables para almacenar la perdida, el numero de muestras, la precision del entrenamiento y la precision de la prueba
    L = 0.0
    N = 0
    Acc = 0.0
    TestAcc = 0.0
    TestN = 0

    # Entrenamiento
    net.train()
    for X, y in train_iter:
        # obtener las predicciones del modelo para las entradas X
        y_hat = net(X)
        # calculo de la perdida
        l = loss(y_hat, y)
        # limpiar los gradientes
        trainer.zero_grad()
        # calcular los gradientes de la perdida media
        l.mean().backward()
        # actualizar los parametros
        trainer.step()
        # sumar la perdida total
        L += l.sum()
        # sumar el numero de muestras
        N += l.numel()
        Acc += accuracy(y_hat, y)

    # Evaluación
    net.eval()
    with torch.no_grad():
        for X, y in test_iter:
            y_hat = net(X)
            TestN += y.numel()
            TestAcc += accuracy(y_hat, y)

    print(f'epoch {epoch + 1}, loss {(L/N):f}, '
          f'train accuracy {(Acc/N):f}, test accuracy {(TestAcc/TestN):f}')

epoch 1, loss 4.626585, train accuracy 0.009804, test accuracy 0.005529
epoch 2, loss 4.626545, train accuracy 0.009804, test accuracy 0.005529
epoch 3, loss 4.626492, train accuracy 0.009804, test accuracy 0.005529
epoch 4, loss 4.626464, train accuracy 0.009804, test accuracy 0.005529
epoch 5, loss 4.626407, train accuracy 0.009804, test accuracy 0.005529
epoch 6, loss 4.626363, train accuracy 0.009804, test accuracy 0.005529
epoch 7, loss 4.626310, train accuracy 0.009804, test accuracy 0.005529
epoch 8, loss 4.626290, train accuracy 0.009804, test accuracy 0.005529
epoch 9, loss 4.626254, train accuracy 0.009804, test accuracy 0.005529
epoch 10, loss 4.626193, train accuracy 0.009804, test accuracy 0.005529


Se observa que los resultados obtenidos en base a la red construida son extremadamente bajos. El modelo no logra generalizar, dando una precisión muy baja. Por cada epoca la funcion de perdida se mantiene casi igual.

Es necesario implementar una arquitectura diferente para lograr resolver el presente problema.